$10,000$ MC replications ($s=10000$), $n=200, 400, 1000$,  
$E[Y|X] = \theta_0^2 X + \theta_0 X^2$, $\theta_0 = 5/4$, $X \sim N(\mu,1)$, $\mu=1$ in (a), $\mu=-1$ in (b).  
$\text{var}(Y|X)$ is constant.

Moments: with $\rho(X,\theta) = 2\theta X + X^2$,  
$(a): \mathbb{E}[\rho(X,\theta_0)(Y-\theta^2 X - \theta X^2)]$  
$(b): \mathbb{E}[\rho(X,\theta)(Y-\theta^2 X - \theta X^2)]$

Do GMM with initial values $-4, -2, 0, 2$ randomly.

In [1]:
import numpy as np
from scipy import optimize

In [28]:
# DGP
def my_dgp(n, mu):
    """
    n = sample size, mu = 1 or -1.
    """
    theta_0 = 5/4
    X = np.random.normal(size=n, loc=mu, scale=1)
    if len(X.shape) == 1:
        X = X.reshape(n,1)
    U = np.random.normal(size=n, loc=0, scale=1) * X
    Y = (theta_0**2)*X + theta_0 * (X**2)
    
    return Y, X

# GMM
def my_gmm(Y,X, moment, init):
    """
    moment = 'a' or 'b'
    """
    n,_ = X.shape
    
    # moment
    rho = lambda X,theta: 2*theta*X + X**2
    moment_a = lambda Y,X,theta: rho(X, 5/4) * (Y - (theta**2)*X - theta*(X**2))
    moment_b = lambda Y,X,theta: rho(X, theta) * (Y - (theta**2)*X - theta*(X**2))
    if moment == 'a':
        mom=moment_a
    elif moment == 'b':
        mom=moment_b
    else:
        print("moment should be 'a' or 'b'")
                                                  
    # objective function
    def obj(theta):
        g = lambda theta: mom(Y,X, theta).mean()
        J = lambda theta: n*(g(theta)**2)
        
        theta1 = optimize.fmin(J, init, disp=False)
        e = mom(Y,X,theta1)
        omega = (e-e.mean()).T@(e-e.mean())/n  # centered
        
        obj = n * g(theta)**2 / omega
        return obj
    coef = optimize.fmin(obj, init)
    return coef

In [29]:
# mu = 1
mu = 1
n = [200, 400, 1000]
moment = ['a', 'b']
init = [-4, 2, 0, 2]



In [30]:
# mu = -1
mu = -1
n = [200, 400, 1000]
moment = ['a', 'b']
init = [-4, 2, 0, 2]